# Create accessible route

In [ ]:
# Select where to run notebook: "azure" or "local"
my_run = "azure"

In [ ]:
import set_path

import numpy as np
import pandas as pd

import shapely.ops as so
import shapely.geometry as sg
import geopandas as gpd

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import contextily as cx
import folium

import networkx as nx
import momepy

import plot_utils
import poly_utils

import settings as st
if my_run == "azure":
    import config_azure as cf
elif my_run == "local":
    import config as cf

## User inputs

In [ ]:
# Profile settings
max_curb_height = 0.04  # m
min_sidewalk_width = 0.80  # m
walk_bike_preference = 'walk' # choose between walk and bike

In [ ]:
# # Route 1 (Nieuwmarkt/Lastage)
origin_coords = [52.36961721766481, 4.9003201135539225]  # Pentagon 43
dest_coords = [52.372116213810244, 4.900804660058156]  # Nieuwmarkt metro stop

# # Route 2 (Nieuwmarkt/Lastage)
# origin_coords = [52.37006044454489, 4.910663945243775]  # Nieuwe Herengracht 261
# dest_coords = [52.36789258235766, 4.905669275986852]  # Mr. Visserplein, tram stop

# # Route 3 (Nieuwmarkt/Lastage)
# origin_coords = [52.36670252662803, 4.904332947310768]  # Jonas Daniël Meijerplein 50
# dest_coords = [52.36789258235766, 4.905669275986852]  # Waterlooplein, metro ingang (TODO: correct location of entrance in network)

# Route 4 (Osdorp-Midden)
# origin_coords = [52.36383090783878, 4.79453871695722]  # Rapenburgerplein
# dest_coords = [52.362275420653454, 4.78770521434305]  # Baden Powellweg, busstop

# # Route 5 (Osdorp-Midden)
# origin_coords = [52.362744704886246, 4.793565374557849]  # Willemskerkestraat 7
# dest_coords = [52.36002038891118, 4.790022542072676]  # Saaftingestraat, bus stop

# # Route 6 (Osdorp-Midden)
# origin_coords = [52.35428532356661, 4.796174777612867]  # Lederambachtstraat 260
# dest_coords = [52.35800870737117, 4.798853466754499]  # Hoekenes, tram/busstop

# # Route 7 (Osdorp-Midden)
# origin_coords = [52.34992615251687, 4.796073119507634]  # Klaas Katerstraat 154
# dest_coords = [52.3529798396042, 4.79968797061476]  # Koos Vorrinkweg, bus

## Get network data

In [ ]:
os.system('sudo blobfuse /home/azureuser/cloudfiles/code/blobfuse/sidewalk --tmp-path=/mnt/resource/blobfusetmp --config-file=/home/azureuser/cloudfiles/code/blobfuse/fuse_connection_sidewalk.cfg -o attr_timeout=3600 -o entry_timeout=3600 -o negative_timeout=3600 -o allow_other -o nonempty')

In [ ]:
# Get network with widths,crossings and public transport stops
df_raw = gpd.read_file(cf.output_basic_network_including_crossings)

In [ ]:
# Select relevant columns
df = df_raw[['length', 'obstacle_free_width_float', 'width_fill', 'sidewalk_id', 'crossing',
       'walk_bike_connection', 'walk_public_transport_stop_connection',
       'public_transport_stop', 'bikepath_id',
       'crossing_type', 'curb_height_max', 'stop_type', 'stop_name',
       'stop_placement_type', 'wheelchair_accessible', 'geometry']]
df = df.rename(columns={'walk_public_transport_stop_connection': 'walk_pt_connection'})
df.head(3)

In [ ]:
# Check if missings values are as expected
print(len(df))
df.isna().sum()

## Set hard limits

In [ ]:
df['include'] = 1

### Prepare to drop crossings based max curb height

In [ ]:
# Check if there are any curb height crossings without max height
df[df['crossing_type'] == 'curb_height']['curb_height_max'].value_counts(dropna=False)

In [ ]:
# Don't include crossings with curbs that are too high
df.loc[df['curb_height_max'] > max_curb_height, 'include'] = 0

In [ ]:
# Check if the right amount of paths are included
print(df['curb_height_max'].value_counts(dropna=False))
print(df['include'].value_counts(dropna=False))

### Prepare to drop paths based on min width

In [ ]:
# TODO determine if this is final and if we want to keep it here
# Give crossings a width
df.loc[df['crossing'] == 'Yes', 'obstacle_free_width_float'] = st.width_6
df.loc[df['crossing'] == 'Yes', 'width_fill'] = 4

# Give bike paths a width
df.loc[~df['bikepath_id'].isnull(), 'obstacle_free_width_float'] = st.width_5
df.loc[~df['bikepath_id'].isnull(), 'width_fill'] = 4

# Give walk bike connections a width
df.loc[df['walk_bike_connection'] == 'Yes', 'obstacle_free_width_float'] = st.width_5
df.loc[df['walk_bike_connection'] == 'Yes', 'width_fill'] = 4

# Give walk public transport stop connections a width if unknown
df.loc[(df['walk_pt_connection'] == 'Yes') & df['obstacle_free_width_float'].isnull(), 'width_fill'] = 4
df.loc[(df['walk_pt_connection'] == 'Yes') & df['obstacle_free_width_float'].isnull(), 'obstacle_free_width_float'] = st.width_2

In [ ]:
# Check if there are any remaining paths without width
df.loc[(df['obstacle_free_width_float'].isnull()) & (df['public_transport_stop'] == 'No')]

In [ ]:
# Don't include paths that are too narrow
df.loc[df['obstacle_free_width_float'] < min_sidewalk_width, 'include'] = 0

In [ ]:
# Check if the right amount of paths are included
print(df['obstacle_free_width_float'].value_counts())
print(df['include'].value_counts(dropna=False))

## Create objective

In [ ]:
# Define weight (combination of objectives)
df['my_weight'] = df['length']

### Include preference to prevent crossings

In [ ]:
crossing_weight_factor = 1.4
df.loc[df['crossing'] == 'Yes', 'my_weight'] = df['length'] * crossing_weight_factor

### Include bike/pedestrian preference

In [ ]:
walk_bike_preference_weight_factor = 0.6

if walk_bike_preference == 'walk':
    df.loc[df['bikepath_id'].notna(), 'my_weight'] = df['length'] * walk_bike_preference_weight_factor
elif walk_bike_preference == 'bike':
    df.loc[df['sidewalk_id'].notna(), 'my_weight'] = df['length'] * walk_bike_preference_weight_factor

In [ ]:
# Check weights
df['my_weight'].plot()

In [ ]:
# Normalize weights
df['my_weight'] = df['my_weight'] /df['my_weight'].abs().max()

## Store final dataframe

In [ ]:
df.to_file(cf.output_final_network, driver='GPKG')

## Create graphs

In [ ]:
# Select accessible paths to include
df_sel = df[df['include'] == 1]

In [ ]:
df.shape

In [ ]:
df_sel.shape

In [ ]:
# Create graph based on max height and min width
G_sel = momepy.gdf_to_nx(df_sel, approach="primal", multigraph=True)

# Create full graph
G = momepy.gdf_to_nx(df, approach="primal", multigraph=True)

### Take connectivity into account

In [ ]:
# Check sizes of connected components for full graph
[len(c) for c in sorted(nx.connected_components(G), key=len, reverse=True)][:20]

In [ ]:
# Check sizes of connected components for graph based on max height and min width
[len(c) for c in sorted(nx.connected_components(G_sel), key=len, reverse=True)][:20]

In [ ]:
# Get subgraphs
S = [G.subgraph(c).copy() for c in sorted(nx.connected_components(G), key=len, reverse=True)]
S_sel = [G_sel.subgraph(c).copy() for c in sorted(nx.connected_components(G_sel), key=len, reverse=True)]

In [ ]:
# Compose final graph based on largest subgraphs
G_con = nx.compose(S[0], S[1])
G_sel_con = nx.compose(S_sel[0], S_sel[1])
G_sel_con = nx.compose(G_sel_con, S_sel[2])
G_sel_con = nx.compose(G_sel_con, S_sel[3])
G_sel_con = nx.compose(G_sel_con, S_sel[4])

## Visualize network

In [ ]:
# Create dataframes with full, connected and final network
G_df = momepy.nx_to_gdf(G, points=False, lines=True)
G_con_df = momepy.nx_to_gdf(G_con, points=False, lines=True)
G_sel_con_df = momepy.nx_to_gdf(G_sel_con, points=False, lines=True)

In [ ]:
G_df.shape

In [ ]:
G_con_df.shape

In [ ]:
G_sel_con_df.shape

In [ ]:
# # Set center of map
# area_coords = [52.375664816888225, 4.8632280955697995]

# # Set to map coordinate reference system
# G_df_show = G_df.to_crs("EPSG:4326")
# G_con_df_show = G_con_df.to_crs("EPSG:4326")
# G_sel_con_df_show = G_sel_con_df.to_crs("EPSG:4326")

# # Create basic map
# my_map = folium.Map(
#     location=area_coords,
#     tiles='cartodbpositron',
#     min_zoom=10, max_zoom=20, zoom_start=13,
#     zoom_control=True, control_scale=True, control=False
#     )

# # Add complete network
# folium.GeoJson(data=G_df_show, style_function=lambda x: {"color": "lightgray"}).add_to(my_map)

# # Overlay network without hard limits (but with taking connectivity into account)
# folium.GeoJson(data=G_con_df_show[G_con_df_show['crossing'] == 'No'], style_function=lambda x: {"color": "lightblue"}).add_to(my_map)
# folium.GeoJson(data=G_con_df_show[G_con_df_show['crossing'] == 'Yes'], style_function=lambda x: {"color": "pink"}).add_to(my_map)
# folium.GeoJson(data=G_con_df_show[~G_con_df_show['bikepath_id'].isnull()], style_function=lambda x: {"color": "lightgreen"}).add_to(my_map)

# # Overlay final network (with hard limits and taking connectivity into account)
# folium.GeoJson(data=G_sel_con_df_show[G_sel_con_df_show['crossing'] == 'No'], style_function=lambda x: {"color": "blue"}).add_to(my_map)  
# folium.GeoJson(data=G_sel_con_df_show[G_sel_con_df_show['crossing'] == 'Yes'], style_function=lambda x: {"color": "purple"}).add_to(my_map)
# folium.GeoJson(data=G_sel_con_df_show[~G_sel_con_df_show['bikepath_id'].isnull()], style_function=lambda x: {"color": "darkgreen"}).add_to(my_map)

# # Add public transport stop nodes icluding information to the network
# gdf_ptf = df.loc[df['public_transport_stop'] == 'Yes']
# feature_names = gdf_ptf.columns.tolist()
# feature_names.remove('geometry')
# color_column = 'wheelchair_accessible'
# tooltip = plot_utils.gen_tooltip(feature_names, feature_names)
# folium.GeoJson(gdf_ptf, tooltip=tooltip, marker=folium.Marker(icon=folium.Icon(icon='solid fa-code-merge', prefix='fa')), style_function=lambda feature: {"markerColor": 'green' if
#                                                                                                                                             feature["properties"][color_column] == 'Yes' else 
#                                                                                                                                             'red' if feature["properties"][color_column] == 'No' else
#                                                                                                                                             'orange'}).add_to(my_map)

In [ ]:
# # Store map
# my_map.save(cf.network_map_final)

## Get route

In [ ]:
# Define origin and destination
df_coords = pd.DataFrame({"coordinates": ["origin", "destination"],
       "latitude": [origin_coords[0], dest_coords[0]],
       "longitude": [origin_coords[1], dest_coords[1]]})

# Create geodataframe
gdf_coords = gpd.GeoDataFrame(
    df_coords, geometry=gpd.points_from_xy(df_coords.longitude, df_coords.latitude), crs="EPSG:4326"
)
gdf_coords = gdf_coords.to_crs(st.CRS)
gdf_coords = gdf_coords[['coordinates', 'geometry']]

# Get origin and destination location
origin_point = gdf_coords.loc[gdf_coords['coordinates'] == 'origin', 'geometry'].values[0]
dest_point = gdf_coords.loc[gdf_coords['coordinates'] == 'destination', 'geometry'].values[0]

In [ ]:
# Get origin and destination node location - in full network
origin_node_loc = so.nearest_points(origin_point, sg.MultiPoint(list(G_con.nodes)))[1]
dest_node_loc = so.nearest_points(dest_point, sg.MultiPoint(list(G_con.nodes)))[1]

# Get origin and destination node - in full network
origin_node = (origin_node_loc.x, origin_node_loc.y)
dest_node = (dest_node_loc.x, dest_node_loc.y)
print(origin_node)
print(dest_node)

In [ ]:
# Get origin and destination node location - in accessible network
origin_node_sel_loc = so.nearest_points(origin_point, sg.MultiPoint(list(G_sel_con.nodes)))[1]
dest_node_sel_loc = so.nearest_points(dest_point, sg.MultiPoint(list(G_sel_con.nodes)))[1]

# Get origin and destination node - in accessible network
origin_node_sel = (origin_node_sel_loc.x, origin_node_sel_loc.y)
dest_node_sel = (dest_node_sel_loc.x, dest_node_sel_loc.y)
print(origin_node_sel)
print(dest_node_sel)

In [ ]:
heuristic = 'dijkstra'
# heuristic = 'a_star'

elif heuristic =='dijkstra':
    # Get shortest path between origin and destination, based on length (in full network)
    my_path_length = nx.shortest_path(G_con, origin_node, dest_node, weight='length')
    # Get 'shortest' path between origin and destination, based on custom weight (in accessible network)
    my_path_weight = nx.shortest_path(G_sel_con, origin_node_sel, dest_node_sel, weight='my_weight')

if heuristic == 'a_star':
    # Get shortest path between origin and destination, based on length (in full network)
    my_path_length = nx.astar_path(G_con, origin_node, dest_node, heuristic=poly_utils.dist, weight='length')
    # Get 'shortest' path between origin and destination, based on custom weight (in accessible network)
    my_path_weight = nx.astar_path(G_sel_con, origin_node_sel, dest_node_sel, heuristic=poly_utils.dist, weight='my_weight')

In [ ]:
# Put shortest path (based on length) in a dataframe
G_path_length = nx.subgraph(G_con, my_path_length)
df_path_length = momepy.nx_to_gdf(G_path_length, lines=True, points=False)

In [ ]:
# Put 'shortest' path (based on accessibility weight) in a dataframe
G_path_weight = nx.subgraph(G_sel_con, my_path_weight)
df_path_weight = momepy.nx_to_gdf(G_path_weight, lines=True, points=False)

In [ ]:
# Check lengths of routes (m)
print(df_path_length['mm_len'].sum().round())
print(df_path_weight['mm_len'].sum().round())

## Visualize route

In [ ]:
# Subset network for plotting
my_rad = 70
gdf_coords['buffer'] = gdf_coords['geometry'].buffer(my_rad, cap_style=3)
plot_area = gpd.GeoDataFrame(geometry=[gdf_coords['buffer'][0].union(gdf_coords['buffer'][1])], crs=st.CRS)
df_sub = gpd.sjoin(df, plot_area, how='inner').reset_index()

In [ ]:
df_path_weight

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))

# Network
df_sub.plot(ax=ax, color='lightgrey', linewidth=1)

# 'Shortest' paths
df_path_weight.plot(ax=ax, color='black', linewidth=3)
df_path_weight[~df_path_weight['bikepath_id'].isnull()].plot(ax=ax, color='darkgreen', linewidth=3)
df_path_length.plot(ax=ax, color='grey', linewidth=2)

# Origin and destination location
gdf_coords.head(1).plot(ax=ax, color='blue', markersize=50)
gdf_coords.tail(1).plot(ax=ax, color='red', markersize=50)

# Origin and destination nodes
gpd.GeoSeries([origin_node_sel_loc], crs=st.CRS).plot(ax=ax, color='blue', markersize=20)
gpd.GeoSeries([dest_node_sel_loc], crs=st.CRS).plot(ax=ax, color='red', markersize=20)

# Background
cx.add_basemap(ax=ax, source=cx.providers.CartoDB.Voyager, crs=st.CRS)

# Legend
route_acc = mpatches.Patch(color='black', label='accessible route (pedestrian)')
route_acc_b = mpatches.Patch(color='darkgreen', label='accessible route (bike)')
route = mpatches.Patch(color='grey', label='shortest route')
origin = mpatches.Patch(color='blue', label= 'origin')
dest = mpatches.Patch(color='red', label= 'destination')
plt.legend(handles=[route_acc,route_acc_b,route,origin,dest], loc='lower center')
# plt.legend(handles=[route_acc,origin,dest], loc='lower center')

plt.axis('off')

# plt.savefig('../data/accessible_route.png', bbox_inches='tight')
plt.show()

## Store route

In [ ]:
df_path_weight['length'] = df_path_weight['length'].round(2)
df_path_weight_store = df_path_weight.to_crs('epsg:4326')
df_path_weight_store = df_path_weight_store[['geometry', 'length']]
df_path_points_store = df_path_weight_store['geometry'].get_coordinates()
df_path_points_store = df_path_points_store.reset_index(names='linestring_id')
df_path_points_store

In [ ]:
gdf_coords_store = gdf_coords.to_crs('epsg:4326')
gdf_coords_store = gdf_coords_store[['coordinates', 'geometry']]
gdf_coords_points_store = gdf_coords_store.set_index('coordinates')['geometry'].get_coordinates()
gdf_coords_points_store = gdf_coords_points_store.reset_index()
gdf_coords_points_store

In [ ]:
# Store the data
# df_path_points_store.to_csv('../data/accessible_route.csv', sep=';')
# gdf_coords_points_store.to_csv('../data/accessible_route_start_end.csv', sep=';')